# Email Extraction using Pyhton

### NOTE: Before using your email here, make sure that you have IMAP enabled for your account and allow access to less secure apps.
### NOTE: This code will keep scanning for the keywords mentioned until it finds 5 mails. So make sure that your account has at least 5 recent mails that contain these keywords otherwise scanning all mails will be a legthy process.

#### Exectute all blocks sequentially. DO NOT skip any block.

In [ ]:
import imaplib, email
from email.header import decode_header
from bs4 import BeautifulSoup

In [ ]:
username = 'enter_email_here'       # enter the email you want to use here. Example: xyz@gmail.com
password = 'enter_password_here'    # enter the password for this email here
imap_url = 'imap.gmail.com' # enter the IMAP for your mailing service here. Do not change if you are using Gmail.
words = ['Congrat', 'congrat'] # enter keywords to search here. Default is congrat for congratulatory emails.

In [ ]:
con = imaplib.IMAP4_SSL(imap_url)

In [ ]:
con.login(username,password)

In [ ]:
con.list()

In [ ]:
status, messages = con.select('Inbox')
print(status)

In [ ]:
messageCount = int(messages[0])
print(messageCount)

In [ ]:
def getResult(msg, wordList):
    mailContent= []
    checkWords = wordList
    for response in msg:
        if isinstance(response, tuple):
            mess = email.message_from_bytes(response[1])
            subject = decode_header(mess["Subject"])[0][0]
            from_ = mess.get('From')
            if mess.is_multipart():
                for part in mess.walk():
                    content_type = part.get_content_type()
                    try:
                        body = part.get_payload(decode=True).decode()
                    except:
                        pass
                    if content_type == 'text/plain' :
                        mailContent.append(body)
                    elif content_type == 'text/html' :
                        soup = BeautifulSoup(body, 'html.parser')
                        htmltext = soup.find_all(text=True)
                        mailContent.append(' '.join(htmltext))
            else:
                content_type = mess.get_content_type()
                try:
                    body = mess.get_payload(decode=True).decode()
                except:
                    pass
                if content_type == 'text/plain' :
                    mailContent.append(body)
                elif content_type == 'text/html' :
                    soup = BeautifulSoup(body, 'html.parser')
                    htmltext = soup.find_all(text=True)
                    mailContent.append(' '.join(htmltext))
            
    for i in checkWords:
        if (i in subject) or (i in ''.join(mailContent)):
            return subject
            break

In [ ]:
firstFive = []
for i in range(messageCount,-1,-1):
    res, msg = con.fetch(str(i), "(RFC822)")
    try:
        sub = getResult(msg, words)
        if sub :
            firstFive.append(sub)
    except:
        pass
    if len(firstFive) >= 5:
        break

In [ ]:
print("Subjects of the mails: \n")
print(*firstFive, sep = "\n") 